In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

learning_rate = 0.001
total_epoch = 30
batch_size = 128

# 사람은 글씨를 위에서 아래로 내려가면서 쓰는 경향이 많으니 데이터를
# 윗 첫 줄(픽셀) 을 첫 단계 입력값으로 삼고, 28번째 줄(픽셀)을 28단계 입력값으로 받습니다.
n_input =28
n_step = 28
n_hidden = 128
n_class = 10


# 기존 모델과 다른 점은 입력값 X에 n_step이라는 차원을 하나 추가한 부분입니다.
# RNN은 순서가 있는 데이터를 다루므로 한 번에 입력 받을 개수와 총 몇 단계로 이뤄진 데이터를 받을지를 설정해야 합니다.
# 이를 위해 가로 픽셀 수를 n_input으로, 세로 픽셀 수를 입력 단계인 n_step으로 설정하였습니다.
# 출력값은 MNIST의 분류인 0~9까지 10개의 숫자를 원-핫 인코딩으로 표현하도록 만들었습니다.
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.float32, [None, n_class])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

# RNN의 기본 신경망은 긴 단계의 데이터를 학습할 때 맨 뒤에서는 맨 앞의 정보를 잘 기억하지 못하는 특성이 있습니다.
# 이를 보완하기 위해 다양한 구조가 만들어 졌고, 그중 가장 많이 사용되는 것이 LSTM이라는 신경망입니다.
# GRU는 LSTM과 비슷하지만, 구조가 조금 더 간단한 신경망 아키텍처입니다.
cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)

# dynamic_rnn함수를 이용해 RNN 신경망을 완성합니다.
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

# 원래는 다음과 같이 주어진 단계를 반복하는 과정을 거쳐야 합니다.
# 한 단계를 학습한 뒤 상태를 저장하고, 그 상태를 다음 단계의 입력 상태로 하여 다시 학습합니다.
# 이렇게 주어진 단계만큼 반복하여 상태를 전파하면서 출력값을 만들어가는 것이 RNN의 기본구조입니다.
#states = tf.zeros(batch_size)
#for i in range(n_step):
#    outputs, states = cell(X:[[:, i]], states)
#...

# outputs : [batch_size, n_step, n_hidden]
# -> [n_step, batch_size, n_hidden]
outputs = tf.transpose(outputs, [1, 0, 2])
# -> [batch_size, n_hidden]
outputs = outputs[-1]

model = tf.matmul(outputs, W) + b

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

/usr/local/Cellar/python3/3.6.4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [2]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(total_epoch):
    total_cost = 0
    for i in range(total_batch):
        
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape((batch_size, n_step, n_input))
        
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys})
        total_cost += cost_val
        
    print('Epoch:', '%04d' % (epoch + 1), 'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))
    
print('최적화 완료!')

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

test_batch_size = len(mnist.test.images)

test_xs = mnist.test.images.reshape(test_batch_size, n_step, n_input)
test_ys = mnist.test.labels

print('정확도:', sess.run(accuracy, feed_dict={X: test_xs, Y: test_ys}))


Epoch: 0001 Avg. cost = 0.536
Epoch: 0002 Avg. cost = 0.259
Epoch: 0003 Avg. cost = 0.192
Epoch: 0004 Avg. cost = 0.162
Epoch: 0005 Avg. cost = 0.139
Epoch: 0006 Avg. cost = 0.122
Epoch: 0007 Avg. cost = 0.125
Epoch: 0008 Avg. cost = 0.117
Epoch: 0009 Avg. cost = 0.108
Epoch: 0010 Avg. cost = 0.099
Epoch: 0011 Avg. cost = 0.102
Epoch: 0012 Avg. cost = 0.100
Epoch: 0013 Avg. cost = 0.090
Epoch: 0014 Avg. cost = 0.086
Epoch: 0015 Avg. cost = 0.089
Epoch: 0016 Avg. cost = 0.085
Epoch: 0017 Avg. cost = 0.082
Epoch: 0018 Avg. cost = 0.076
Epoch: 0019 Avg. cost = 0.079
Epoch: 0020 Avg. cost = 0.073
Epoch: 0021 Avg. cost = 0.075
Epoch: 0022 Avg. cost = 0.072
Epoch: 0023 Avg. cost = 0.074
Epoch: 0024 Avg. cost = 0.072
Epoch: 0025 Avg. cost = 0.065
Epoch: 0026 Avg. cost = 0.065
Epoch: 0027 Avg. cost = 0.063
Epoch: 0028 Avg. cost = 0.061
Epoch: 0029 Avg. cost = 0.064
Epoch: 0030 Avg. cost = 0.069
최적화 완료!
정확도: 0.9732
